# Load dependencies

In [1]:
#load dependencies
from sql_conn import connection #SQL connection details and variables
#import neuralnet as nn #functions to form neural network and calculate accuracy and plot accuracy with history
import preproc as pr #functions to preprocess data 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

from sklearn import preprocessing #for one-hot encoding

For the Live SQL Server Authentication, please enter your


# SQL Script

In [2]:
s='''
    
        SELECT DISTINCT
        ST.[StudyID] as [StudyID]
        ,[StudyTitle]
        ,[StudyShortName]
        ,StudyResearchSummary
        ,StudyInclusionCriteria
        ,StudyDesignType
        ,StudyPhases_Concatenated
        ,StudySettings_Concatenated
        ,StudyManagingSpecialty
        ,StudyManagingSpecialty_PrimarySubSpecialty
        ,[ICDChapter]

        FROM(
        SELECT 
        [StudyID]
        ,[ICDChapter]
        from [CPMS_BI].[dbo].[ODP_StudyICD]
        WHERE StudyID is NOT NULL) AS IC

        LEFT JOIN 
        (SELECT 
        StudyID
        ,[StudyTitle]
        ,[StudyShortName]
        ,StudyResearchSummary
        ,StudyInclusionCriteria
        ,StudyDesignType
        ,StudyPhases_Concatenated
        ,StudySettings_Concatenated
        ,StudyManagingSpecialty
        ,StudyManagingSpecialty_PrimarySubSpecialty
        ,Study_IsCommercial
        ,Study_IsDraft

        FROM [CPMS_BI].[dbo].[ODP_Study]
        WHERE 1=1
        ) AS ST
        ON IC.StudyID = ST.StudyID
        WHERE 1=1
        AND Study_IsCommercial = 0
        AND Study_IsDraft=0
    '''

# with open("C:/Users/busaji/Documents/Projects/StudyClassifier-main/dat/sql-read.txt","r") as f:
#     s = f.read()

In [3]:
study_df = pd.read_sql(s, connection)

C:\Users\busaji\AppData\Local\Temp\ipykernel_30796\3074574563.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  study_df = pd.read_sql(s, connection)


In [14]:
study_df.dtypes

StudyID                                        int64
StudyTitle                                    object
StudyShortName                                object
StudyResearchSummary                          object
StudyInclusionCriteria                        object
StudyDesignType                               object
StudyPhases_Concatenated                      object
StudySettings_Concatenated                    object
StudyManagingSpecialty                        object
StudyManagingSpecialty_PrimarySubSpecialty    object
ICDChapter                                    object
dtype: object

In [4]:
#pre-processing
study_df2 = study_df.fillna("not available")

#convert to float
#study_df['StudyResearchSummary'] = study_df['StudyResearchSummary'].astype("string")

In [5]:
study_df2

,StudyID,StudyTitle,StudyShortName,StudyResearchSummary,StudyInclusionCriteria,StudyDesignType,StudyPhases_Concatenated,StudySettings_Concatenated,StudyManagingSpecialty,StudyManagingSpecialty_PrimarySubSpecialty,ICDChapter
0,646,A Randomised Trial of High Dose Therapy in Loc...,RT01,To compare standard 64 Gy dose of conformal ra...,1) Histologically confirmed cancer of the pros...,Interventional,III,Primary Care,Cancer,Prostate Cancer,Neoplasms
1,651,A Randomised trial of Hormone Therapy plus Rad...,PR07,To evaluate any possible benefit from the addi...,Histologically confirmed Prostate cancer withi...,Interventional,III,Primary Care,Cancer,Prostate Cancer,Neoplasms
2,760,The ProtecT Trial - Evaluating the effectivene...,ProtecT,The overall aim is to evaluate the effectivene...,- inclusion for randomisation: men with locali...,Interventional,III,"Primary Care, Secondary Care",Cancer,Prostate Cancer,Neoplasms
3,791,INTERmittent or CONTINuos ENdocrine Therapy Af...,Intercontinental,A phase III randomised trial comparing intermi...,Patients with adenocarcinoma of the prostate w...,Interventional,III,Primary Care,Cancer,Prostate Cancer,Neoplasms
4,798,Intermittent androgen deprivation in patients ...,EORTC 30985,not available,not specified,Interventional,III,"Secondary Care, Tertiary Care",Cancer,Prostate Cancer,Neoplasms
...,...,...,...,...,...,...,...,...,...,...,...
14803,64537,Glycemic and insulin metabolism testing of ara...,Glycemic and insulin metabolism testing of ara...,The significance of dietary fibre in promoting...,• Male or female • Age between 18-65 years (...,Interventional,N/A,Community,Metabolic and Endocrine Disorders,Metabolic and Endocrine Disorders,"Endocrine, nutritional and metabolic diseases"
14804,64764,BEING ACTIVE AFTER SARCOMA: Understanding how ...,BEACON - Sarcoma,Background: Despite the importance of physical...,"To be eligible, potential participants will ha...",Observational,N/A,Tertiary Care,Cancer,Teenage and Young Adult's Cancer,Neoplasms
14805,64776,Emotional support for women experiencing prete...,Emotional support for women experiencing PPROM...,Preterm prelabour rupture of the membranes occ...,• Woman (or birthing person) aged 16 years or ...,Interventional,N/A,"Secondary Care, Tertiary Care",Reproductive Health and Childbirth,General Obstetrics (antenatal and postnatal ca...,"Pregnancy, childbirth and the puerperium"
14806,64793,Cognitive Problems in Ageing and Psychosis,CAP,Background This project seeks to improve the ...,Inclusion criteria (VLOP group) Age > = 60 ye...,Observational,N/A,Secondary Care,Ageing,Ageing,Mental and behavioural disorders


In [6]:
#get study ID & ICD chapter
study_icd = study_df2[["StudyID","ICDChapter"]]

# One-Hot encoding for ICD Chapters

In [7]:
#load onehotencoder
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', 
                    categories='auto', 
                    drop='if_binary')

enc.fit(study_icd)


OneHotEncoder(drop='if_binary', handle_unknown='ignore')

In [ ]:
study_icd

In [8]:
#pandas one-hot encoding

df_encoded = pd.get_dummies(study_icd, dtype=int)

In [9]:
df_encoded.shape
#should be 23 shapes - 22 ICD Chapters and 1 blank

(14808, 23)

In [ ]:
df_encoded

In [10]:
#Merge source data and df_encoded together

data = pd.merge(
    left=study_df2,
    right=df_encoded,
    how='right',
    on='StudyID'
)

In [11]:
data.shape

(19948, 33)

# Pre-processing

In [12]:
#combine text data into one column
data['Text'] = data['StudyTitle'] + " " + data['StudyResearchSummary'] + " " + data['StudyInclusionCriteria']

In [13]:
#Setting StudyID as dataframe index
data.set_index('StudyID', inplace=True)

In [ ]:
data

In [14]:
#drop unrequired fields

data_cln = data.drop(['StudyTitle','StudyResearchSummary','StudyInclusionCriteria',
            'StudyTitle', 'ICDChapter',
          'StudyDesignType','StudyPhases_Concatenated','StudySettings_Concatenated',
          'StudyShortName',
          'StudyManagingSpecialty','StudyManagingSpecialty_PrimarySubSpecialty'], axis=1)

In [ ]:
data_cln

In [15]:
#split dependent and independent features
X = data_cln['Text']
y=data_cln.drop('Text', axis=1)

# TF-IDF Vectorizer

In [21]:
#test code for tf-idf and then multi-class multi-label classifier

vectorizer = TfidfVectorizer(max_features=5000, max_df=0.7, )
vectorizer.fit(X)

TfidfVectorizer(max_df=0.7, max_features=5000)

In [22]:
#split in train and test
# from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)
  
X_train_tfidf = vectorizer.transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Multi-class Multi-Label Classfication

In [23]:
#multi-classifier model and prediction
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression


clf = MultiOutputClassifier(LogisticRegression()).fit(X_train_tfidf, y_train)

In [24]:
#prediction
prediction = clf.predict(X_test_tfidf)
prediction

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [25]:
#evaluation
from sklearn.metrics import accuracy_score
print('Accuracy Score: ', accuracy_score(y_test, prediction))

Accuracy Score:  0.3769423558897243


In [26]:
#random forest
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100, random_state = 1, )
multi_forest = MultiOutputClassifier(forest).fit(X_train_tfidf, y_train)
## Make prediction
dfOutput_multi_forest = multi_forest.predict_proba(X_test_tfidf)

In [30]:
#store prediction and check accuracy

forest_prediction = multi_forest.predict(X_test_tfidf)

In [31]:
print('Accuracy Score: ', accuracy_score(y_test, forest_prediction))

Accuracy Score:  0.3215538847117794


In [28]:
# feature importance from random forest:
import numpy as np
feat_impts = [] 
for clf in multi_forest.estimators_:
    feat_impts.append(clf.feature_importances_)

np.mean(feat_impts, axis=0)

array([2.43989858e-04, 5.72654844e-05, 5.51617222e-04, ...,
       6.26236492e-05, 6.95234487e-05, 3.28897310e-05])

# Function for prediction (TEST) below:

In [ ]:
def multi_label_classification(X_train, y_train, X_test):
    """
    This function builds and evaluates a multi-label classification model using Scikit-Learn.
    It trains a Logistic Regression model, and prints the classification report.
    """
    # Initialize the base classifier
    base_classifier = LogisticRegression(max_iter=1000)
    # Wrap the base classifier with MultiOutputClassifier
    multi_target_classifier = MultiOutputClassifier(base_classifier, n_jobs=-1)
    # Train the multi-label classifier
    multi_target_classifier.fit(X_train, y_train)
    # Predict the labels for the test set
    y_pred = multi_target_classifier.predict(X_test)
    # Evaluate the model
    report = classification_report(y_test, y_pred, target_names=[f'Label {i}' for i in range(y.shape[1])])
    # Print the classification report
    print(report)

In [ ]:
def multi_label_classification(X_train, y_train, X_test, y_test):
       # Convert text data to TF-IDF features
    vectorizer = TfidfVectorizer()
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)
    # Initialize the base classifier
    base_classifier = LogisticRegression(max_iter=1000)
    # Wrap the base classifier with MultiOutputClassifier
    multi_target_classifier = MultiOutputClassifier(base_classifier, n_jobs=-1)
    # Train the multi-label classifier
    multi_target_classifier.fit(X_train_tfidf, y_train)
    # Predict the labels for the test set
    y_pred = multi_target_classifier.predict(X_test_tfidf)
    # Evaluate the model
    report = classification_report(y_test, y_pred, target_names=[f'Label {i}' for i in range(len(y_train[0]))])
    # Print the classification report
    print(report)